#### 00. Install dependancies

In [14]:
%%capture

# Core training libraries
!pip install -q \
    transformers==4.44.2 \
    datasets==2.20.0 \
    tokenizers==0.19.1 \
    accelerate==0.34.2 \
    peft==0.12.0 \
    trl==0.9.6 \
    bitsandbytes==0.43.1 \
    evaluate==0.4.2

# Utilities
!pip install -q \
    numpy \
    pandas \
    scikit-learn \
    rich \
    pyyaml \
    python-dotenv \
    tqdm

# Evaluation (requires pydantic v2)
!pip install -q --upgrade pydantic
!pip install -q google-genai rouge-score

print(" Installation complete!")
print(" All dependencies compatible (pydantic v2 + google-genai)")


## 1. Setting Up Environment Variables (Secrets)

In [15]:

# Create .env file with API key
import os

# Write .env file
# with open('.env', 'w') as f:
#     # Add the secrets if needed
#     f.write('GOOGLE_API_KEY=<api_key_here>\n')
#     f.write('HF_TOKEN=<api_key_here>\n')

# print(" .env file created")

In [16]:
# Verify it's loaded
from dotenv import load_dotenv
load_dotenv()
# Show only key names for security
try:
    with open('.env', 'r') as f:
        print(" Keys in .env file:")
        print("="*60)
        for line in f:
            line = line.strip()
            if line and not line.startswith('#'):
                key = line.split('=')[0]
                value_preview = line.split('=')[1][:10] + "..." if '=' in line else ""
                print(f"  {key} = {value_preview}")
        print("="*60)
except FileNotFoundError:
    print(" .env file not found")

 .env file not found


## 2. Environment & GPU Check

In [17]:
import sys
import torch

print("="*60)
print("ENVIRONMENT CHECK")
print("="*60)
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device capability: {torch.cuda.get_device_capability(0)}")
    print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print(" WARNING: CUDA not available. Training will be VERY slow on CPU.")

print("="*60)

ENVIRONMENT CHECK
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch version: 2.9.0+cpu
CUDA available: False


## 3. Seeds & Determinism

Setting up random seeds for reproducibility. .

In [ ]:
import os
import random
import numpy as np
import torch

SEED = 42

# Set environment variable for Python hash seed
os.environ['PYTHONHASHSEED'] = str(SEED)

# Set seeds
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    # Note: These settings may impact performance
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print(f" Seeds set to {SEED} for reproducibility")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## 4. Hugging Face Login

If you want to push your finetuned adapter to the Hugging Face Hub, uncomment and run the login line below.

Hugging Face token with write permissions. Get one at: https://huggingface.co/settings/tokens

In [ ]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
!hf auth login --token $HF_TOKEN

print("ℹ Hugging Face login skipped. Uncomment login() to push models to Hub.")

## 5. Configuration (Single Source of Truth)

All hyperparameters and settings in one place. **Edit here** to customize your training.

In [22]:
import torch
from pprint import pprint

# Auto-detect compute dtype (BF16 requires compute capability >= 8.0)
use_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8
compute_dtype = torch.bfloat16 if use_bf16 else torch.float16

CONFIG = {
    # Model
    "base_model": "Qwen/Qwen2.5-1.5B-Instruct",
    # Alternative for tighter VRAM: "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    # For GGUF export, prefer: "meta-llama/Llama-3.2-3B-Instruct" or Mistral models

    # Dataset
    "dataset_name": "lavita/AlpaCare-MedInstruct-52k",
    "dataset_split": "train",
    "dataset_subsample": 500,  # Colab-safe: 500 | Local: 1500
    "train_val_split": 0.9,  # 90% train, 10% validation

    # Tokenization
    "max_length": 512,  # Colab: 512 | Local: 1024

    # Training
    "num_train_epochs": 1,
    "max_steps": 250,  # Colab: 250 | Local: 600
    "per_device_train_batch_size": 1,  # Colab: 1 | Local: 2
    "gradient_accumulation_steps": 64,  # Colab: 64 | Local: 32
    "learning_rate": 2e-5,
    "warmup_ratio": 0.03,
    "logging_steps": 10,
    "save_steps": 200,
    "eval_steps": 100,
    "save_total_limit": 2,

    # LoRA
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "lora_target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],

    # Quantization
    "load_in_4bit": True,
    "bnb_4bit_compute_dtype": compute_dtype,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": True,

    # Output
    "output_dir": "outputs/adapter",
    "push_to_hub": False,

    # Generation
    "max_new_tokens": 128,
    "temperature": 0.0,  # Deterministic
    "do_sample": True,

    # HF credentials
    'hf_username': '<your-username>',
    'hub_model_name': 'zuucrew-medical-assistant',
}

# Effective batch size
effective_batch_size = CONFIG["per_device_train_batch_size"] * CONFIG["gradient_accumulation_steps"]

print("="*60)
print("CONFIGURATION (COLAB FREE TIER)")
print("="*60)
pprint(CONFIG)
print("="*60)
print(f"Compute dtype: {compute_dtype}")
print(f"Using BF16: {use_bf16}")
print(f"Effective batch size: {effective_batch_size}")
print("="*60)

CONFIGURATION (COLAB FREE TIER)
{'base_model': 'Qwen/Qwen2.5-1.5B-Instruct',
 'bnb_4bit_compute_dtype': torch.float16,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': True,
 'dataset_name': 'lavita/AlpaCare-MedInstruct-52k',
 'dataset_split': 'train',
 'dataset_subsample': 500,
 'do_sample': True,
 'eval_steps': 100,
 'gradient_accumulation_steps': 64,
 'hf_username': '<your-username>',
 'hub_model_name': 'zuucrew-medical-assistant',
 'learning_rate': 2e-05,
 'load_in_4bit': True,
 'logging_steps': 10,
 'lora_alpha': 32,
 'lora_dropout': 0.05,
 'lora_r': 16,
 'lora_target_modules': ['q_proj',
                         'k_proj',
                         'v_proj',
                         'o_proj',
                         'gate_proj',
                         'up_proj',
                         'down_proj'],
 'max_length': 512,
 'max_new_tokens': 128,
 'max_steps': 250,
 'num_train_epochs': 1,
 'output_dir': 'outputs/adapter',
 'per_device_train_batch_size': 1,
 'push_to_hub